# Pinecone 

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [4]:
pip install -q pinecone

Note: you may need to restart the kernel to use updated packages.


In [68]:
pip install --upgrade pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
pc.list_indexes()

[
    {
        "name": "langchain",
        "metric": "cosine",
        "host": "langchain-jp289co.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    }
]

# Working with pinecone indexes

In [28]:
pc.list_indexes().names()

['langchain']

In [49]:
index_name = "langchain"

if not pc.has_index(index_name):
    print(f'Creating Index {index_name}')
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f'Index {index_name} already exists')

Creating Index langchain


## Delete Index

In [48]:
index_name = "langchain"

if pc.has_index(index_name):
    print(f'Deleting Index {index_name}')
    pc.delete_index(index_name)
    print("Done")
else:
    print(f'Index {index_name} already deleted')

Deleting Index langchain
Done


## Performing operation on Index

In [35]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# Working with Vectors

In [50]:
import random
# generating 5 vectors
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
index = pc.Index(index_name)
index.upsert(vectors=zip(ids,vectors))

{'upserted_count': 5}

## Update Vectors

In [40]:
index.upsert(vectors=[('c',[0.5] * 1536)])

{'upserted_count': 1}

# Fetch Vectors

In [42]:
index.fetch(ids=['c', 'd'])

FetchResponse(namespace='', vectors={'c': Vector(id='c', values=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 

# Delete Vector

In [43]:
index.delete(ids=['b', 'c'])

{}

In [44]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3,
 'vector_type': 'dense'}

In [46]:
# fetching non existing vector
index.fetch(ids=['x'])

FetchResponse(namespace='', vectors={}, usage={'read_units': 1})

# Query

In [51]:
query_vector = [random.random() for _ in range(1536)]

In [52]:
# will retrieve ids of 3 most similar vectors along with similarity score
index.query(
    vector = query_vector,
    top_k = 3,
    include_values=False
)

{'matches': [{'id': 'c', 'score': 0.748815, 'values': []},
             {'id': 'e', 'score': 0.748297572, 'values': []},
             {'id': 'a', 'score': 0.747524083, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}

# Namespace

In [53]:
# adding 3 vectors to first-namespace
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids,vectors), namespace="first-namespace")

{'upserted_count': 3}

In [54]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3}},
 'total_vector_count': 8,
 'vector_type': 'dense'}

In [55]:
index.fetch(ids=['x'], namespace="first-namespace")

FetchResponse(namespace='first-namespace', vectors={'x': Vector(id='x', values=[0.305166513, 0.127974555, 0.448198617, 0.40673992, 0.0586064979, 0.059577737, 0.0254814215, 0.620314062, 0.248792946, 0.089644894, 0.32663542, 0.851494133, 0.436741948, 0.336131096, 0.89353168, 0.408809215, 0.985404313, 0.23422499, 0.722419798, 0.913641036, 0.0539909452, 0.821655631, 0.375693262, 0.299481064, 0.985198, 0.212604687, 0.3718189, 0.117923848, 0.744728506, 0.632505, 0.110824242, 0.546807349, 0.159709543, 0.241411448, 0.826256633, 0.0743033, 0.450311095, 0.229321823, 0.567636, 0.910315216, 0.797227442, 0.721906602, 0.85980612, 0.151943341, 0.471302688, 0.302086, 0.583878756, 0.793727934, 0.788680911, 0.0293269027, 0.441502243, 0.485853761, 0.562536359, 0.432394475, 0.531806588, 0.572813809, 0.518336833, 0.869849443, 0.130013034, 0.179327413, 0.796764433, 0.265195578, 0.72299391, 0.983243167, 0.523660183, 0.859932, 0.915377617, 0.148849636, 0.435212433, 0.542380095, 0.419511318, 0.98678124, 0.4577

In [56]:
index.delete(ids=['x'], namespace="first-namespace")

{}

In [57]:
# delete all record from a namespace
index.delete(delete_all=True, namespace="first-namespace")

{}

In [58]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5,
 'vector_type': 'dense'}